In [44]:
#start
import gspread #type:ignore NOQA
import imaplib
import email
import json

#connecting to GSeets and selecting the file
sa = gspread.service_account(filename="sa_creds.json")

#read credentials from json
with open("ym_creds.json", "r") as f:
    jmCreds = json.load(f)

groceryBrandDoneEmails = [] #list to write to GSheets
def groceryBrandEmails(GroceryBrand: str, emailList):
    for i in emailList:
        f = [GroceryBrand]
        f.append(i)
        groceryBrandDoneEmails.append(f)

sh = sa.open("Grocery")
#selecting sheets:
wksEmails = sh.worksheet("Emails")
wksSummaryMaxima = sh.worksheet("MaximaSummarized")

#getting data from wksEmails sheet
# wksEmails.get()
maximaAlreadyIn = [i[1] for i in wksEmails.get() if 'Maxima' in i[0]]

In [2]:
# Connect to the Yahoo Mail IMAP server
imap_server = imaplib.IMAP4_SSL("imap.mail.yahoo.com")
# Login to the account
imap_server.login(jmCreds["email"], jmCreds["password"])
# Select the "Inbox" folder
imap_server.select("Inbox")

# Search for all emails: status, email_ids = imap_server.search(None, "ALL")
# Search for all emails from MAXIMA
status, email_ids = imap_server.search(None, "FROM noreply.code.provider@maxima.lt")
maximaEmails = email_ids[0].decode().split()

maximaToAdd = [i for i in maximaEmails if i not in maximaAlreadyIn]

In [3]:
maximaToAdd

['5468', '5482', '5495', '5497']

In [4]:
for i in maximaToAdd:
    status, email_data = imap_server.fetch(i, "(RFC822)")
    email_message = email.message_from_bytes(email_data[0][1])

In [5]:
#get message bu ID
status, email_data = imap_server.fetch("5468", "(RFC822)")
email_message = email.message_from_bytes(email_data[0][1])

In [6]:
msg1 = email_message.get_payload()[0]
msg1body = msg1.get_payload(decode=True)
msg2 = email_message.get_payload()[0]
msg2body = msg2.get_payload(decode=True)
msg3 = email_message.get_payload()[0]
msg3body = msg3.get_payload(decode=True)

In [7]:
# print(msg1.get_payload(decode=True), msg2.get_payload(decode=True))
print(msg2body.decode())

<html><body>


<html>

<body>

    <div class="previewText" style="display:none; max-height: 0px; overflow: hidden">
            <p>Apsipirkimo suma: 23.89 EUR</p>
            <br />
                <p>Kvito nuolaidų suma: 9.30 EUR</p>
                <br />
                <p> MAXIMOS PINIGŲ likutis: 0.24 EUR </p>
            <p>
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;

In [8]:
# Disconnect from the server
imap_server.close()
imap_server.logout()

('BYE', [b'IMAP4rev1 Server logging out'])

In [9]:
x = msg2body.decode().split('\r\n')
for e, i in enumerate(x):
    print(e, x[e])
# x[10]

0 <html><body>
1 
2 
3 <html>
4 
5 <body>
6 
7     <div class="previewText" style="display:none; max-height: 0px; overflow: hidden">
8             <p>Apsipirkimo suma: 23.89 EUR</p>
9             <br />
10                 <p>Kvito nuolaidų suma: 9.30 EUR</p>
11                 <br />
12                 <p> MAXIMOS PINIGŲ likutis: 0.24 EUR </p>
13             <p>
14                 &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
15                 &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
16                 &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
17                 &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwn

In [86]:
receiptSummaryData = [[]]
def readMaximaReceiptSummary(EmailId: str, receipt):
    for i in range(1, 11):
        receiptSummaryData[0].append('')
    receiptSummaryData[0][0] = EmailId
    for i in receipt:
        #Receipt_id
        if '<pre>Kvitas ' in i:
            receiptSummaryData[0][1] = i.split()[1]
        #BankReceipt_id
        if 'KVITO NR' in i:
            receiptSummaryData[0][2] = i.split()[2]
        #Receipt_Document#
        if 'DOKUMENTO NR' in i:
            receiptSummaryData[0][3] = i.split()[2]
        #RRN
        if 'RRN:' in i:
            receiptSummaryData[0][4] = i.split()[1]
        #Date
        if 'LTF NM ' in i:
            receiptSummaryData[0][5] = f'{i.split()[3]}-{i.split()[4]}-{i.split()[5]}'
        #Time
        if 'LTF NM ' in i:
            receiptSummaryData[0][6] = i.split()[6]
        #Time
        if 'MAXIMA LT' in i:
            receiptSummaryData[0][7] = 'MAXIMA LT'


In [87]:
# receiptSummaryData = [['','','','','','','','','','','']]
readMaximaReceiptSummary('1234' ,x)
# receiptSummaryData

In [88]:
wksSummaryMaxima.append_rows(receiptSummaryData)

{'spreadsheetId': '10-mjgW6dVIMLh5iVe6NbyIbI_N3XZ0mOFGlxilp4W0s',
 'tableRange': 'MaximaSummarized!A1:K2',
 'updates': {'spreadsheetId': '10-mjgW6dVIMLh5iVe6NbyIbI_N3XZ0mOFGlxilp4W0s',
  'updatedRange': 'MaximaSummarized!A3:J3',
  'updatedRows': 1,
  'updatedColumns': 10,
  'updatedCells': 10}}

In [29]:
for i in x:
    if 'LTF NM ' in i:
        print(i.split())
        print(i.split()[6])

['LTF', 'NM', '0000003BEFA4', '2022', '12', '19', '19:57:49']
19:57:49


In [24]:
t.split()[2]

'458107'